In [94]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.stats import t

<h2> We choose the following nest : measured vs flat 

Options 1 and 2 are measured, options 3, 4 and 5 are flat </h2>

In [95]:
# Define log-likelihood function for telephone data
# beta will be beta = ["ASC_2", "ASC_3", "ASC_4", "ASC_5", "BETA_COST", "lambda_measured", "lambda_flat"]

def log_likelihood_telephone(beta, data):
    # Define utility functions
    data['U_1'] = beta[4] * data['cost1'] 
    data['U_2'] = beta[0] + beta[4] * data['cost2'] 
    data['U_3'] = beta[1] + beta[4] * data['cost3']
    data['U_4'] = beta[2] + beta[4] * data['cost4']
    data['U_5'] = beta[3] + beta[4] * data['cost5']

    # Avoid numerical issues
    data.loc[data['U_1'] > 1000, 'U_1'] = 1000
    data.loc[data['U_2'] > 1000, 'U_2'] = 1000
    data.loc[data['U_3'] > 1000, 'U_3'] = 1000
    data.loc[data['U_4'] > 1000, 'U_4'] = 1000
    data.loc[data['U_5'] > 1000, 'U_5'] = 1000
    data.loc[data['U_1'] < -1000, 'U_1'] = -1000
    data.loc[data['U_2'] < -1000, 'U_2'] = -1000
    data.loc[data['U_3'] < -1000, 'U_3'] = -1000
    data.loc[data['U_4'] < -1000, 'U_4'] = -1000
    data.loc[data['U_5'] < -1000, 'U_5'] = -1000
    
    # Calculate logsum for nests with > 1 alt
    data['logsum_measured'] = np.log(data['avail1'] * np.exp(data['U_1'] / beta[5])
                                        + data['avail2'] * np.exp(data['U_2'] / beta[5])
                                        + (1 - data['avail1']) * (1 - data['avail2']))
    data['logsum_flat'] = np.log(data['avail3'] * np.exp(data['U_3'] / beta[6])
                                    + data['avail4'] * np.exp(data['U_4'] / beta[6])
                                    + data['avail5'] * np.exp(data['U_5'] / beta[6])
                                    + (1 - data['avail3']) * (1 - data['avail4']) * (1 - data['avail5']))
    
    # Nest probabilities
    data['P_nest_measured'] = np.exp(beta[5] * data['logsum_measured']) / \
                                 (np.exp(beta[5] * data['logsum_measured']) 
                                  + np.exp(beta[6] * data['logsum_flat']))
    data['P_nest_flat'] = 1 - data['P_nest_measured']
    
    # Within nest probabilities for nests with > 1 alt
    data['P_1_in_measured'] = data['avail1'] * np.exp(data['U_1'] / beta[5]) / \
                                (data['avail1'] * np.exp(data['U_1'] / beta[5]) 
                                 + data['avail2'] * np.exp(data['U_2'] / beta[5]))
    data['P_2_in_measured'] = 1 - data['P_1_in_measured']

    data['P_3_in_flat'] = data['avail3'] * np.exp(data['U_3'] / beta[6]) / \
                            (data['avail3'] * np.exp(data['U_3'] / beta[6])
                                + data['avail4'] * np.exp(data['U_4'] / beta[6])
                                + data['avail5'] * np.exp(data['U_5'] / beta[6]))
    data['P_4_in_flat'] = data['avail4'] * np.exp(data['U_4'] / beta[6]) / \
                            (data['avail3'] * np.exp(data['U_3'] / beta[6])
                                + data['avail4'] * np.exp(data['U_4'] / beta[6])
                                + data['avail5'] * np.exp(data['U_5'] / beta[6]))
    data['P_5_in_flat'] = 1 - data['P_3_in_flat'] - data['P_4_in_flat']
    
    # Full probabilities
    data['P_1'] = data['P_nest_measured'] * data['P_1_in_measured']
    data['P_2'] = data['P_nest_measured'] * data['P_2_in_measured']
    data['P_3'] = data['P_nest_flat'] * data['P_3_in_flat']
    data['P_4'] = data['P_nest_flat'] * data['P_4_in_flat']
    data['P_5'] = data['P_nest_flat'] * data['P_5_in_flat']
    
    # Calculate probability for chosen alternative for each row
    data['P'] = (data['choice'] == 1) * data['P_1'] + \
                (data['choice'] == 2) * data['P_2'] + \
                (data['choice'] == 3) * data['P_3'] + \
                (data['choice'] == 4) * data['P_4'] + \
                (data['choice'] == 5) * data['P_5']
    
    # Calculate log-likelihood
    LL = data['P'].apply(np.log).sum()
    
    return -LL  # We minimize negative log-likelihood


In [96]:
def estimate_nested_logit(data, beta_initial, beta_names, log_likelihood_function):
    """
    Estimate parameters for a nested logit model using maximum likelihood estimation.

    Args:
    - data (DataFrame): Input dataset containing variables needed for the model.
    - beta_initial (array-like): Initial guess for model parameters.
    - beta_names (list): Names of model parameters.
    - log_likelihood_function (function): Function that calculates the log-likelihood of the model. 

    Returns:
    - result (OptimizeResult): Result object from scipy.optimize.minimize containing optimization results.
    - se (array-like): Robust asymptotic standard errors of parameter estimates.
    - t_stat (array-like): t-statistics of parameter estimates.
    - p_value (array-like): p-values of parameter estimates.
    """

    # Run the model
    result = minimize(log_likelihood_function, x0 = beta_initial, args = data, method='BFGS')

    # Calculate Hessian matrix
    hessian_inv = result.hess_inv

    # Calculate robust asymptotic standard errors
    se = np.sqrt(np.diag(hessian_inv))

    # Calculate t-statistics
    t_stat = result.x / se

    # Calculate p-values
    p_value = (1 - t.cdf(np.abs(t_stat), len(data) - len(beta_initial))) * 2

    # Create DataFrame to store results
    results_df = pd.DataFrame({
        "Parameter": beta_names,
        "Estimate": result.x,
        "Robust Asymptotic SE": se,
        "t-statistic": t_stat,
        "p-value": p_value
    })

    print("Optimization Results:")
    print(results_df)

    return result, se, t_stat, p_value


In [97]:
# Load data
data = pd.read_csv('./data/telephone.dat', sep='\t')

data_cost = data[['choice', 'avail1', 'avail2', 'avail3', 'avail4', 'avail5',
                   'cost1', 'cost2', 'cost3', 'cost4', 'cost5']]

In [98]:
# Determine number of individuals in the data
N = len(data['choice'])

# Define model parameters
beta = np.array([0, 0, 0, 0, 0, 1, 1])
# lambda_n = 1 / mu_n is a measure of the degree of independence in unobserved utility among
# the alternatives in nest n.
# It should be between 0 and 1 with lambda_n = 1 indicating full independence.
beta_names = ["ASC_2", "ASC_3", "ASC_4", "ASC_5", "BETA_COST", "lambda_measured", "lambda_flat"]
fixed_params = {}  # No fixed parameters in this example


In [99]:
# Estimate parameters
result, se, t_stat, p_value = estimate_nested_logit(data, beta, beta_names, log_likelihood_telephone)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:590: RuntimeWarning: invalid value encountered in subtract
  df =

Optimization Results:
         Parameter  Estimate  Robust Asymptotic SE  t-statistic       p-value
0            ASC_2  0.229560              0.048430     4.739995  2.916229e-06
1            ASC_3  0.441990              0.129858     3.403636  7.277404e-04
2            ASC_4 -2.206832              1.829185    -1.206457  2.283095e-01
3            ASC_5  0.388151              0.186663     2.079415  3.817544e-02
4        BETA_COST -0.204925              0.017377   -11.792574  0.000000e+00
5  lambda_measured  0.374129              0.057819     6.470691  2.678899e-10
6      lambda_flat  3.515004              0.425237     8.265994  1.776357e-15


In [100]:
data.head(15)

,choice,area,users,inc,age0,age1,age2,age3,age4,age5,...,P_2_in_measured,P_3_in_flat,P_4_in_flat,P_5_in_flat,P_1,P_2,P_3,P_4,P_5,P
0,2,1,2,4,0,0,0,1,1,0,...,0.653734,0.652312,0.0,0.347688,0.162171,0.306171,0.346807,0.0,0.184851,0.306171
1,3,1,1,2,0,0,0,0,0,0,...,0.348803,0.658500,0.0,0.341500,0.332939,0.178333,0.321827,0.0,0.166900,0.321827
2,1,2,2,4,0,0,0,2,0,0,...,0.558640,0.730142,0.0,0.269858,0.227949,0.288520,0.353046,0.0,0.130485,0.227949
3,3,2,2,3,1,0,0,0,2,0,...,0.512351,0.741585,0.0,0.258415,0.237237,0.249254,0.380810,0.0,0.132698,0.380810
4,3,2,6,3,1,3,2,1,0,2,...,0.807709,0.696059,0.0,0.303941,0.100266,0.421162,0.333115,0.0,0.145458,0.333115
5,3,2,2,2,1,0,0,2,0,0,...,0.807709,0.710894,0.0,0.289106,0.031311,0.131521,0.595137,0.0,0.242030,0.595137
6,3,2,1,1,0,0,0,0,0,0,...,0.558640,0.724941,0.0,0.275059,0.235333,0.297866,0.338403,0.0,0.128398,0.338403
7,1,2,2,5,1,0,0,1,1,0,...,0.329170,0.743813,0.0,0.256187,0.348999,0.171250,0.356845,0.0,0.122906,0.348999
8,5,2,4,4,1,1,0,0,2,0,...,0.807709,0.362849,0.0,0.637151,0.011994,0.050378,0.340218,0.0,0.597411,0.597411
9,3,2,2,5,0,0,0,2,0,0,...,0.807709,0.545822,0.0,0.454178,0.037169,0.156126,0.440317,0.0,0.366388,0.440317


<h2>

- Comprendre pourquoi lambda_flat est plus grand que 1 

- Regarder s'il faut afficher les covariance matrix etc... 

- Regarder comment récupérer les distributions depuis l'estimation finale 

- Essayer d'appliquer IB/DIB à la distribution récupérée. 

</h2>